In [ ]:
%matplotlib widget
import math, numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam
tf.config.set_visible_devices([], 'GPU')             # disable GPU for this notebook

signal_index=2
expt_index=4

def get_features( T: np.ndarray ) -> np.ndarray:
    features = []
    t, tL = T-T[0], T[-1]-T[0]
    features.append(t/tL)
    for ibase, npow in [ (2,12) ]: # , (3,8), (5,5), (6,4), (7,3) ]:
        for ip in range(1,npow+1):
            base = tL/math.pow( ibase, ip )
            features.append( np.mod(t,base)/base )
    return np.stack(features, axis=1)

data_dir =  "/explore/nobackup/projects/ilab/data/astrotime/demo"
ckp_file = f"{data_dir}/embed_time_predict.e{expt_index}.s{signal_index}.weights.h5"
data=np.load( f'{data_dir}/jordan_data.npz',allow_pickle=True )
signals = data['signals']
times = data['times']
#binary_times = data['binary_times']

# X = binary_times[signal_index].astype(np.float32)
X: np.ndarray = get_features( times[signal_index] )
Y: np.ndarray = signals[signal_index]
T: np.ndarray = times[signal_index]
validation_split: int = int(0.8*X.shape[0])

Xtrain=X[:validation_split]
Xval=X[validation_split:]
Ytrain=Y[:validation_split]
Yval=Y[validation_split:]

In [ ]:
def create_small_model(nfeatures: int, dropout_frac: float):
    binary_times_input = tf.keras.Input(shape=(nfeatures,), name="time_features")

    x = tf.keras.layers.Dense(512, activation='tanh')(binary_times_input)
    x = tf.keras.layers.Dropout(dropout_frac)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dense(512, activation='tanh')(x)
    x = tf.keras.layers.Dropout(dropout_frac)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dense(512, activation='tanh')(x)
    x = tf.keras.layers.Dropout(dropout_frac)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dense(512, activation='tanh')(x)
    x = tf.keras.layers.BatchNormalization()(x)

    outputs = tf.keras.layers.Dense(1, activation='linear')(x)
    model = tf.keras.Model(inputs=binary_times_input, outputs=outputs)
    return model

In [ ]:
optimizer = Adam( learning_rate=0.001, name='adam' )
model = create_small_model(X.shape[1],0.5)
model.compile(optimizer=optimizer, loss='mae')
model.load_weights( ckp_file )
p0=model.predict(Xtrain,batch_size=256)
p1=model.predict(Xval,batch_size=256)

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(T,Y,label='truth')
plt.plot(T[:validation_split],p0[:,0],label='train prediction')
plt.plot(T[validation_split:],p1[:,0],label='val prediction')
plt.title(f'Signal {signal_index}')
plt.legend()
plt.tight_layout()
plt.show()